In [1]:
import numpy as np
from torch.utils import data
from torch.utils.data import Dataset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import _LRScheduler

torch.backends.cudnn.deterministic = True

class Dataset(data.Dataset):
    def __init__(self, X1, Y1):
        self.X1 = X1
        self.Y1 = Y1

    def __len__(self):
        return len(self.X1)

    def __getitem__(self, index):
        x = self.X1[index]
        y1 = self.Y1[index]
        return x, y1

train_features=np.load("../extracted_features/FS-change3/input_features_free_train.npy")
labels_stress_train=np.load("../extracted_features/FS-change3/label_free_train.npy")


test_features=np.load("../extracted_features/FS-change3/input_features_free_test.npy")
labels_stress_test=np.load("../extracted_features/FS-change3/label_free_test.npy")


val_features=np.load("../extracted_features/FS-change3/input_features_free_val.npy")
labels_stress_val=np.load("../extracted_features/FS-change3/label_free_val.npy")


In [2]:
print(labels_stress_train.shape,labels_stress_train)

(1121,) [0. 0. 0. ... 1. 1. 1.]


In [3]:
batch_size=20
train_data = Dataset(train_features, labels_stress_train)
test_data=Dataset(test_features,labels_stress_test)
val_data=Dataset(val_features,labels_stress_val)
train_data_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_data_loader=DataLoader(test_data,shuffle=True,batch_size=batch_size)
test_data_loader=DataLoader(val_data,shuffle=True,batch_size=batch_size)

In [4]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [5]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



In [6]:
class Classifier(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.input_dim = 8
        self.hidden_dim = 200
        self.layer_dim = 1
        self.output_dim=1
        self.rnn = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)
        self.sig=nn.Sigmoid()
    
    def forward(self, x):
        h0, c0 = self.init_hidden(x)
        out, (hn, cn) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out=self.sig(out)
        return out
    
    def init_hidden(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim)
        return [t.cuda() for t in (h0, c0)]

In [7]:
feature_size = 8
shared_layer_size = 512
LR = 0.0001
epoch = 200
model=Classifier()
model.cuda()
iterations_per_epoch = len(train_data_loader)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=LR/100))

In [8]:
from sklearn.metrics import accuracy_score
best_acc1 = 0
modelname=[]
truth=[]
preds=[]

for it in range(epoch+1):
    model.train()
    total=len(train_data_loader)*batch_size
    train_loss = 0.
    for minibatch in train_data_loader:
        X, Y1  = minibatch
        X=X.cuda()
        Y1=Y1.cuda()
        output = model(X.float())
        output=output.squeeze(1)
        loss = loss_func(output, Y1.float())
        Y_hat1 = torch.ge(output, 0.5).float()
        train_loss += loss.item()
        truth.extend(Y1.tolist())
        preds.extend(Y_hat1.tolist())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sched.step()
    trainacc1=accuracy_score(truth,preds)
    train_loss /= total
    print("EPOCH ",it)
    print('Train : Loss: {:.4f}, Train acc1 : {:.4f}'.format(train_loss,trainacc1))
    
    count=0
    val_loss= 0.
    truth=[]
    preds=[]
    total=len(val_data_loader)*batch_size
    model.eval()    
    for minibatch in val_data_loader:
        X_valid, Y1_valid  = minibatch
        X_valid=X_valid.cuda()
        Y1_valid=Y1_valid.cuda()
        output_val = model(X_valid.float())
        output_val=output_val.squeeze(1)
        loss = loss_func(output_val, Y1_valid.float())
        Y_hat1_val = torch.ge(output_val, 0.5).float()
        val_loss += loss.item()
        truth.extend(Y1_valid.tolist())
        preds.extend(Y_hat1_val.tolist())
    valacc1=accuracy_score(truth,preds)
    val_loss /= total
    print('Val : Loss: {:.4f}, Val acc1 : {:.4f}'.format(val_loss,valacc1))
    if valacc1 >= best_acc1:
        best_acc1 = valacc1
        best_state = model.state_dict()
        print('Best validation accuracy1 ', best_acc1)

EPOCH  0
Train : Loss: 0.0340, Train acc1 : 0.5709
Val : Loss: 0.0384, Val acc1 : 0.3053
Best validation accuracy1  0.30526315789473685
EPOCH  1
Train : Loss: 0.0332, Train acc1 : 0.5728
Val : Loss: 0.0389, Val acc1 : 0.3000
EPOCH  2
Train : Loss: 0.0328, Train acc1 : 0.5828
Val : Loss: 0.0401, Val acc1 : 0.3105
Best validation accuracy1  0.3105263157894737
EPOCH  3
Train : Loss: 0.0322, Train acc1 : 0.5957
Val : Loss: 0.0407, Val acc1 : 0.3158
Best validation accuracy1  0.3157894736842105
EPOCH  4
Train : Loss: 0.0321, Train acc1 : 0.5988
Val : Loss: 0.0421, Val acc1 : 0.3000
EPOCH  5
Train : Loss: 0.0316, Train acc1 : 0.6011
Val : Loss: 0.0433, Val acc1 : 0.3000
EPOCH  6
Train : Loss: 0.0313, Train acc1 : 0.6003
Val : Loss: 0.0441, Val acc1 : 0.2947
EPOCH  7
Train : Loss: 0.0309, Train acc1 : 0.6011
Val : Loss: 0.0446, Val acc1 : 0.2947
EPOCH  8
Train : Loss: 0.0310, Train acc1 : 0.6102
Val : Loss: 0.0463, Val acc1 : 0.3158
Best validation accuracy1  0.3157894736842105
EPOCH  9
Train

EPOCH  83
Train : Loss: 0.0150, Train acc1 : 0.8185
Val : Loss: 0.0681, Val acc1 : 0.3895
EPOCH  84
Train : Loss: 0.0156, Train acc1 : 0.8124
Val : Loss: 0.0720, Val acc1 : 0.4105
EPOCH  85
Train : Loss: 0.0149, Train acc1 : 0.8085
Val : Loss: 0.0699, Val acc1 : 0.4368
EPOCH  86
Train : Loss: 0.0152, Train acc1 : 0.8185
Val : Loss: 0.0665, Val acc1 : 0.4474
Best validation accuracy1  0.4473684210526316
EPOCH  87
Train : Loss: 0.0143, Train acc1 : 0.8291
Val : Loss: 0.0678, Val acc1 : 0.4211
EPOCH  88
Train : Loss: 0.0147, Train acc1 : 0.8131
Val : Loss: 0.0702, Val acc1 : 0.4053
EPOCH  89
Train : Loss: 0.0136, Train acc1 : 0.8284
Val : Loss: 0.0710, Val acc1 : 0.4158
EPOCH  90
Train : Loss: 0.0151, Train acc1 : 0.8185
Val : Loss: 0.0674, Val acc1 : 0.4368
EPOCH  91
Train : Loss: 0.0132, Train acc1 : 0.8352
Val : Loss: 0.0687, Val acc1 : 0.4368
EPOCH  92
Train : Loss: 0.0146, Train acc1 : 0.8223
Val : Loss: 0.0667, Val acc1 : 0.4474
Best validation accuracy1  0.4473684210526316
EPOCH  9

EPOCH  167
Train : Loss: 0.0021, Train acc1 : 0.9161
Val : Loss: 0.1120, Val acc1 : 0.4737
EPOCH  168
Train : Loss: 0.0024, Train acc1 : 0.9169
Val : Loss: 0.1110, Val acc1 : 0.4737
EPOCH  169
Train : Loss: 0.0021, Train acc1 : 0.9199
Val : Loss: 0.1074, Val acc1 : 0.4842
Best validation accuracy1  0.4842105263157895
EPOCH  170
Train : Loss: 0.0027, Train acc1 : 0.9146
Val : Loss: 0.0995, Val acc1 : 0.5158
Best validation accuracy1  0.5157894736842106
EPOCH  171
Train : Loss: 0.0027, Train acc1 : 0.9207
Val : Loss: 0.1127, Val acc1 : 0.4737
EPOCH  172
Train : Loss: 0.0084, Train acc1 : 0.8696
Val : Loss: 0.1027, Val acc1 : 0.4789
EPOCH  173
Train : Loss: 0.0033, Train acc1 : 0.9115
Val : Loss: 0.1041, Val acc1 : 0.4632
EPOCH  174
Train : Loss: 0.0024, Train acc1 : 0.9169
Val : Loss: 0.1078, Val acc1 : 0.4842
EPOCH  175
Train : Loss: 0.0020, Train acc1 : 0.9207
Val : Loss: 0.1084, Val acc1 : 0.4789
EPOCH  176
Train : Loss: 0.0019, Train acc1 : 0.9199
Val : Loss: 0.1102, Val acc1 : 0.478

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

modeltest=Classifier()
modeltest.load_state_dict(best_state)
modeltest.cuda()
modeltest.eval()
truth=[]
preds=[]
for minibatch in test_data_loader:
            X_test, Y1_test  = minibatch
            X_test=X_test.cuda()
            Y1_test=Y1_test.cuda()
            output_test = modeltest(X_test.float())
            output_test=output_test.squeeze(1)
            prediction = torch.ge(output_test, 0.5).float()
            truth.extend(Y1_test.tolist())
            preds.extend(prediction.tolist())
acc=accuracy_score(truth,preds)
# print(truth,preds)
tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
f1score=f1_score(truth, preds)
precision=precision_score(truth, preds)
recall=recall_score(truth,preds)
roc=roc_auc_score(truth,preds)
specificity=tn/(tn+fp)

print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))


0.42 0.41 0.38 0.46 0.42 0.39


In [10]:
for i in range(len(modelname)):
    modeltest=Classifier()
    modeltest.load_state_dict(best_state)
    modeltest.cuda()
    modeltest.eval()
    truth=[]
    preds=[]
    for minibatch in test_data_loader:
                X_test, Y1_test  = minibatch
                X_test=X_test.cuda()
                Y1_test=Y1_test.cuda()
                output_test = modeltest(X_test.float())
                output_test=output_test.squeeze(1)
                prediction = torch.ge(output_test, 0.5).float()
                truth.extend(Y1_test.tolist())
                preds.extend(prediction.tolist())
    acc=accuracy_score(truth,preds)
    # print(truth,preds)
    tn, fp, fn, tp = confusion_matrix(truth, preds).ravel()
    f1score=f1_score(truth, preds)
    precision=precision_score(truth, preds)
    recall=recall_score(truth,preds)
    roc=roc_auc_score(truth,preds)
    specificity=tn/(tn+fp)

    print('{:.2f} {:.2f} {:.2f} {:.2f} {:.2f} {:.2f}'.format(acc,f1score,precision,recall,roc,specificity))